Запросить курсы любой валюты за последний год. Сохранить полученные данные в базу данных Mongo


Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.

Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>

In [1]:
import requests
from zeep import Client
from pymongo import MongoClient
import pandas as pd
import re

Инициализация БД:

In [2]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['lesson_5']
collection = db.currency

In [3]:
URL = 'https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
client = Client(URL)

Forcing soap:address location to HTTPS
Forcing soap:address location to HTTPS


In [4]:
def date_rate(date):
    
    date_rate = {}
    
    try:
                
        currency_list = client.service.GetCursOnDate(date)._value_1._value_1
        currency = currency_list[8]
        
        nom = currency['ValuteCursOnDate']['Vnom']
        rate = currency['ValuteCursOnDate']['Vcurs']
            
        date_rate['date'] = pd.to_datetime(date)
        date_rate['rate'] = float(rate / nom)
        date_rate['name'] = re.findall('[а-яА-Я]+\ [а-яА-Я]*', currency['ValuteCursOnDate']['Vname'])[0]

        return date_rate
        
    except :
        
        pass

    
def min_max_search(date1, date2):
    
    min_max_dict = {}
    
    pipeline = [{'$match' : {'$and': [ {'date': {'$gte': date1} }, {'date': {'$lte': date2} }]}},  
                {'$sort': { 'rate': 1 } }]
    col_list = list(collection.aggregate(pipeline))

    min_max_dict['date_min_rate'] = col_list[0]['date'].strftime('%d %B %Y')
    min_max_dict['min_rate'] = col_list[0]['rate']
    min_max_dict['date_max_rate'] = col_list[-1]['date'].strftime('%d %B %Y')
    min_max_dict['max_rate'] = col_list[-1]['rate']
    min_max_dict['name'] = col_list[0]['name']
    
    return min_max_dict

In [5]:
year_rate_list = []
for date in pd.date_range(end=pd.datetime.today(), periods=365).strftime('%Y-%m-%d').tolist():
    year_rate_list.append(date_rate(date))

In [6]:
collection.insert_many(year_rate_list)
pass

Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.

In [7]:
DATA_1 = '2018-11-02'
DATA_2 = '2019-01-04'

In [8]:
date1 = pd.to_datetime(DATA_1)
date2 = pd.to_datetime(DATA_2)
res = min_max_search(date1, date2)

In [9]:
print(f'Валюта: {res["name"]}\nВыгодно было купить {res["date_min_rate"]}, а продать {res["date_max_rate"]}.\n\
Прибыль: {(res["max_rate"] / res["min_rate"] - 1) * 100 :.2f}%')

Валюта: Гонконгский доллар
Выгодно было купить 03 November 2018, а продать 29 December 2018.
Прибыль: 6.09%
